# 04 — Tableau Ready Dataset: ASEAN Carbon Emission (2000–2024)

Notebook ini menyiapkan dataset final untuk visualisasi di Tableau.

Tujuan:
- Mengunci definisi metrik di Python agar konsisten
- Menghasilkan file CSV siap pakai untuk dashboard Tableau
- Mengurangi kebutuhan calculated field kompleks di Tableau

Output utama:
- data/tableau/trend_4y_long.csv
  Dataset long-format untuk tren 4-tahunan (average annual)

- data/tableau/ranking_change.csv
  Dataset untuk ranking perubahan start vs end period (average annual)

Catatan metodologi:
- Grouping 4-tahunan di-anchor ke 2000
- Tren dan perbandingan hanya memakai grup 4-tahunan yang lengkap
- Agregasi 4-tahunan memakai rata-rata tahunan (average annual)
- Periode pembanding default: 2000–2003 vs 2020–2023 (year_group 2000 vs 2020)


In [1]:
import pandas as pd
import numpy as np
from pathlib import Path


## Load Data

Dataset dipanggil dari file processed lokal agar konsisten dengan notebook sebelumnya.


In [2]:
data_path = "data/process/owid_co2_asean_2000_2024.csv"
df = pd.read_csv(data_path)
df.shape


(250, 79)

In [3]:
sorted(df["country"].unique().tolist()), int(df["year"].min()), int(df["year"].max())


(['Brunei',
  'Cambodia',
  'Indonesia',
  'Laos',
  'Malaysia',
  'Myanmar',
  'Philippines',
  'Singapore',
  'Thailand',
  'Vietnam'],
 2000,
 2024)

## Standardize Column Names

Nama kolom dirapikan agar konsisten untuk metrik total dan sumber emisi.


In [4]:
rename_map = {
    "co2": "co2_total",
    "coal_co2": "co2_coal",
    "oil_co2": "co2_oil",
    "gas_co2": "co2_gas",
    "cement_co2": "co2_cement",
    "flaring_co2": "co2_flaring",
}
df = df.rename(columns=rename_map)
df.columns


Index(['country', 'year', 'iso_code', 'population', 'gdp', 'co2_cement',
       'cement_co2_per_capita', 'co2_total', 'co2_growth_abs',
       'co2_growth_prct', 'co2_including_luc', 'co2_including_luc_growth_abs',
       'co2_including_luc_growth_prct', 'co2_including_luc_per_capita',
       'co2_including_luc_per_gdp', 'co2_including_luc_per_unit_energy',
       'co2_per_capita', 'co2_per_gdp', 'co2_per_unit_energy', 'co2_coal',
       'coal_co2_per_capita', 'consumption_co2', 'consumption_co2_per_capita',
       'consumption_co2_per_gdp', 'cumulative_cement_co2', 'cumulative_co2',
       'cumulative_co2_including_luc', 'cumulative_coal_co2',
       'cumulative_flaring_co2', 'cumulative_gas_co2', 'cumulative_luc_co2',
       'cumulative_oil_co2', 'cumulative_other_co2', 'energy_per_capita',
       'energy_per_gdp', 'co2_flaring', 'flaring_co2_per_capita', 'co2_gas',
       'gas_co2_per_capita', 'ghg_excluding_lucf_per_capita', 'ghg_per_capita',
       'land_use_change_co2', 'land_use

## Select Core Columns

Kolom inti dipilih untuk kebutuhan Tableau.


In [5]:
cols = [
    "country",
    "year",
    "population",
    "co2_total",
    "co2_per_capita",
    "co2_coal",
    "co2_oil",
    "co2_gas",
    "co2_cement",
    "co2_flaring",
]

missing_cols = [c for c in cols if c not in df.columns]
missing_cols


[]

In [6]:
df_core = df[cols].copy()
df_core.shape


(250, 10)

## Create 4-Year Group and Aggregate (Average Annual)

Year group di-anchor ke 2000.
Hanya grup lengkap (4 tahun) yang dipakai untuk output Tableau.


In [7]:
df_core["year_group"] = 2000 + ((df_core["year"] - 2000) // 4) * 4

group_counts = (
    df_core.groupby(["country", "year_group"])
    .size()
    .reset_index(name="n_years")
)

df_4y = (
    df_core.groupby(["country", "year_group"], as_index=False)
    .mean(numeric_only=True)
)

df_4y_full = df_4y.merge(group_counts, on=["country", "year_group"], how="left")
df_4y_complete = df_4y_full[df_4y_full["n_years"] == 4].copy()

df_4y_full.shape, df_4y_complete.shape


((70, 12), (60, 12))

## Output 1: Trend Dataset (Long Format)

Dataset ini cocok untuk Tableau:
- line chart tren
- small multiples per negara
- metric selector dengan filter

Skema:
- country
- year_group
- metric
- value
- unit
- aggregation_type


In [8]:
metric_map = {
    "co2_total": ("CO2 Total", "tonnes", "average_annual"),
    "co2_per_capita": ("CO2 per Capita", "tonnes_per_person", "average_annual"),
    "co2_coal": ("CO2 from Coal", "tonnes", "average_annual"),
    "co2_oil": ("CO2 from Oil", "tonnes", "average_annual"),
    "co2_gas": ("CO2 from Gas", "tonnes", "average_annual"),
    "co2_cement": ("CO2 from Cement", "tonnes", "average_annual"),
    "co2_flaring": ("CO2 from Flaring", "tonnes", "average_annual"),
}

available_metrics = [m for m in metric_map.keys() if m in df_4y_complete.columns]

trend_long = df_4y_complete[["country", "year_group"] + available_metrics].melt(
    id_vars=["country", "year_group"],
    var_name="metric_key",
    value_name="value"
)

trend_long["metric"] = trend_long["metric_key"].map(lambda x: metric_map[x][0])
trend_long["unit"] = trend_long["metric_key"].map(lambda x: metric_map[x][1])
trend_long["aggregation_type"] = trend_long["metric_key"].map(lambda x: metric_map[x][2])

trend_long = trend_long.drop(columns=["metric_key"])
trend_long = trend_long.sort_values(["metric", "country", "year_group"]).reset_index(drop=True)

trend_long.head(10)


,country,year_group,value,metric,unit,aggregation_type
0,Brunei,2000,5.76725,CO2 Total,tonnes,average_annual
1,Brunei,2004,6.65425,CO2 Total,tonnes,average_annual
2,Brunei,2008,8.62075,CO2 Total,tonnes,average_annual
3,Brunei,2012,8.22250,CO2 Total,tonnes,average_annual
4,Brunei,2016,8.96175,CO2 Total,tonnes,average_annual
5,Brunei,2020,11.90150,CO2 Total,tonnes,average_annual
6,Cambodia,2000,2.15975,CO2 Total,tonnes,average_annual
7,Cambodia,2004,2.69600,CO2 Total,tonnes,average_annual
8,Cambodia,2008,4.71325,CO2 Total,tonnes,average_annual
9,Cambodia,2012,6.52150,CO2 Total,tonnes,average_annual


## Output 2: Ranking Change Dataset (Start vs End Period)

Dataset ini cocok untuk Tableau:
- bar chart ranking perubahan
- table perubahan start-end
- scatter change total vs change per capita

Skema:
- country
- metric
- unit
- aggregation_type
- start_period
- end_period
- start_value
- end_value
- change_value
- change_pct


In [9]:
start_period = int(df_4y_complete["year_group"].min())
end_period = int(df_4y_complete["year_group"].max())
start_period, end_period


(2000, 2020)

In [10]:
ranking_metrics = ["co2_total", "co2_per_capita"] + [m for m in available_metrics if m.startswith("co2_") and m not in ["co2_total", "co2_per_capita"]]
ranking_metrics = [m for m in ranking_metrics if m in df_4y_complete.columns]

start_df = df_4y_complete[df_4y_complete["year_group"] == start_period][["country"] + ranking_metrics].copy()
end_df = df_4y_complete[df_4y_complete["year_group"] == end_period][["country"] + ranking_metrics].copy()

chg = start_df.merge(end_df, on="country", suffixes=("_start", "_end"))

rows = []
for m in ranking_metrics:
    start_col = f"{m}_start"
    end_col = f"{m}_end"

    out = pd.DataFrame({
        "country": chg["country"],
        "metric_key": m,
        "start_period": start_period,
        "end_period": end_period,
        "start_value": chg[start_col],
        "end_value": chg[end_col],
    })
    out["change_value"] = out["end_value"] - out["start_value"]
    out["change_pct"] = np.where(
        out["start_value"].abs() > 0,
        out["change_value"] / out["start_value"] * 100,
        np.nan
    )
    rows.append(out)

ranking_change = pd.concat(rows, ignore_index=True)

ranking_change["metric"] = ranking_change["metric_key"].map(lambda x: metric_map[x][0] if x in metric_map else x)
ranking_change["unit"] = ranking_change["metric_key"].map(lambda x: metric_map[x][1] if x in metric_map else "unknown")
ranking_change["aggregation_type"] = ranking_change["metric_key"].map(lambda x: metric_map[x][2] if x in metric_map else "average_annual")

ranking_change = ranking_change.drop(columns=["metric_key"])
ranking_change = ranking_change.sort_values(["metric", "change_value"], ascending=[True, False]).reset_index(drop=True)

ranking_change.head(10)


,country,start_period,end_period,start_value,end_value,change_value,change_pct,metric,unit,aggregation_type
0,Indonesia,2000,2020,311.25950,694.15725,382.89775,123.015603,CO2 Total,tonnes,average_annual
1,Vietnam,2000,2020,65.42750,336.71200,271.28450,414.633755,CO2 Total,tonnes,average_annual
2,Malaysia,2000,2020,134.79750,273.41800,138.62050,102.836106,CO2 Total,tonnes,average_annual
3,Thailand,2000,2020,178.84300,269.09300,90.25000,50.463255,CO2 Total,tonnes,average_annual
4,Philippines,2000,2020,70.06475,145.91750,75.85275,108.260930,CO2 Total,tonnes,average_annual
5,Myanmar,2000,2020,9.60000,31.82625,22.22625,231.523438,CO2 Total,tonnes,average_annual
6,Laos,2000,2020,1.10150,21.70925,20.60775,1870.880617,CO2 Total,tonnes,average_annual
7,Cambodia,2000,2020,2.15975,21.01075,18.85100,872.832504,CO2 Total,tonnes,average_annual
8,Singapore,2000,2020,38.90900,49.20050,10.29150,26.450179,CO2 Total,tonnes,average_annual
9,Brunei,2000,2020,5.76725,11.90150,6.13425,106.363518,CO2 Total,tonnes,average_annual


## Output 3: Decomposition Dataset (Top Countries)

Dataset ini cocok untuk Tableau:
- stacked bar decomposition perubahan
- fokus pada top N negara berdasarkan perubahan CO2 total

Skema:
- country
- source
- start_period
- end_period
- start_value
- end_value
- change_value
- unit
- aggregation_type


In [11]:
top_n = 5

rank_total = ranking_change[ranking_change["metric"] == "CO2 Total"].copy()
top_countries = rank_total.sort_values("change_value", ascending=False)["country"].head(top_n).tolist()
top_countries


['Indonesia', 'Vietnam', 'Malaysia', 'Thailand', 'Philippines']

In [12]:
sources = ["co2_coal", "co2_oil", "co2_gas", "co2_cement", "co2_flaring"]
sources = [s for s in sources if s in df_4y_complete.columns]

start_src = df_4y_complete[df_4y_complete["year_group"] == start_period][["country"] + sources].copy()
end_src = df_4y_complete[df_4y_complete["year_group"] == end_period][["country"] + sources].copy()

src = start_src.merge(end_src, on="country", suffixes=("_start", "_end"))
src = src[src["country"].isin(top_countries)].copy()

decomp_rows = []
for s in sources:
    out = pd.DataFrame({
        "country": src["country"],
        "source_key": s,
        "start_period": start_period,
        "end_period": end_period,
        "start_value": src[f"{s}_start"],
        "end_value": src[f"{s}_end"],
    })
    out["change_value"] = out["end_value"] - out["start_value"]
    decomp_rows.append(out)

decomp = pd.concat(decomp_rows, ignore_index=True)
decomp["source"] = decomp["source_key"].map(lambda x: metric_map[x][0] if x in metric_map else x)
decomp["unit"] = decomp["source_key"].map(lambda x: metric_map[x][1] if x in metric_map else "unknown")
decomp["aggregation_type"] = decomp["source_key"].map(lambda x: metric_map[x][2] if x in metric_map else "average_annual")
decomp = decomp.drop(columns=["source_key"])
decomp = decomp.sort_values(["country", "source"]).reset_index(drop=True)

decomp.head(10)


,country,start_period,end_period,start_value,end_value,change_value,source,unit,aggregation_type
0,Indonesia,2000,2020,17.42575,30.16350,12.73775,CO2 from Cement,tonnes,average_annual
1,Indonesia,2000,2020,67.92250,351.82400,283.90150,CO2 from Coal,tonnes,average_annual
2,Indonesia,2000,2020,7.82450,3.35425,-4.47025,CO2 from Flaring,tonnes,average_annual
3,Indonesia,2000,2020,58.04500,85.15525,27.11025,CO2 from Gas,tonnes,average_annual
4,Indonesia,2000,2020,160.04200,223.66000,63.61800,CO2 from Oil,tonnes,average_annual
5,Malaysia,2000,2020,6.94800,11.74575,4.79775,CO2 from Cement,tonnes,average_annual
6,Malaysia,2000,2020,12.14100,90.83800,78.69700,CO2 from Coal,tonnes,average_annual
7,Malaysia,2000,2020,2.77825,3.57400,0.79575,CO2 from Flaring,tonnes,average_annual
8,Malaysia,2000,2020,52.29000,94.75200,42.46200,CO2 from Gas,tonnes,average_annual
9,Malaysia,2000,2020,60.64050,72.50875,11.86825,CO2 from Oil,tonnes,average_annual


## Save Outputs for Tableau

File disimpan ke folder data/tableau.


In [13]:
out_dir = Path("data/tableau")
out_dir.mkdir(parents=True, exist_ok=True)

trend_path = out_dir / "trend_4y_long.csv"
ranking_path = out_dir / "ranking_change.csv"
decomp_path = out_dir / "decomposition_top_countries.csv"

trend_long.to_csv(trend_path, index=False)
ranking_change.to_csv(ranking_path, index=False)
decomp.to_csv(decomp_path, index=False)

trend_path.as_posix(), ranking_path.as_posix(), decomp_path.as_posix()


('data/tableau/trend_4y_long.csv',
 'data/tableau/ranking_change.csv',
 'data/tableau/decomposition_top_countries.csv')

## Quick Checks

Bagian ini memastikan file output memiliki struktur yang sesuai untuk Tableau.


In [14]:
trend_long.shape, ranking_change.shape, decomp.shape


((420, 6), (70, 10), (25, 9))

In [15]:
trend_long.head(5)


,country,year_group,value,metric,unit,aggregation_type
0,Brunei,2000,5.76725,CO2 Total,tonnes,average_annual
1,Brunei,2004,6.65425,CO2 Total,tonnes,average_annual
2,Brunei,2008,8.62075,CO2 Total,tonnes,average_annual
3,Brunei,2012,8.22250,CO2 Total,tonnes,average_annual
4,Brunei,2016,8.96175,CO2 Total,tonnes,average_annual


In [16]:
ranking_change.head(5)


,country,start_period,end_period,start_value,end_value,change_value,change_pct,metric,unit,aggregation_type
0,Indonesia,2000,2020,311.25950,694.15725,382.89775,123.015603,CO2 Total,tonnes,average_annual
1,Vietnam,2000,2020,65.42750,336.71200,271.28450,414.633755,CO2 Total,tonnes,average_annual
2,Malaysia,2000,2020,134.79750,273.41800,138.62050,102.836106,CO2 Total,tonnes,average_annual
3,Thailand,2000,2020,178.84300,269.09300,90.25000,50.463255,CO2 Total,tonnes,average_annual
4,Philippines,2000,2020,70.06475,145.91750,75.85275,108.260930,CO2 Total,tonnes,average_annual


In [17]:
decomp.head(5)


,country,start_period,end_period,start_value,end_value,change_value,source,unit,aggregation_type
0,Indonesia,2000,2020,17.42575,30.16350,12.73775,CO2 from Cement,tonnes,average_annual
1,Indonesia,2000,2020,67.92250,351.82400,283.90150,CO2 from Coal,tonnes,average_annual
2,Indonesia,2000,2020,7.82450,3.35425,-4.47025,CO2 from Flaring,tonnes,average_annual
3,Indonesia,2000,2020,58.04500,85.15525,27.11025,CO2 from Gas,tonnes,average_annual
4,Indonesia,2000,2020,160.04200,223.66000,63.61800,CO2 from Oil,tonnes,average_annual


## Tableau Usage Notes

Dashboard yang umum:
- Trend: filter metric, line chart year_group vs value, color by country
- Ranking: filter metric, bar chart country vs change_value, label change_pct
- Decomposition: stacked bar country vs change_value, stack by source

Definisi metrik:
- Semua nilai pada trend_4y_long dan ranking_change adalah average annual pada grup 4-tahunan
- start_period dan end_period di ranking_change mengikuti grup 4-tahunan lengkap pertama dan terakhir
